In [1]:
import paperdecoding
import numpy as np
import pandas as pd
import glob
import re

In [5]:
import pdfplumber

# All papers and reports to text #

In [14]:
for path in glob.glob('sa2papers/*.pdf'):
    pages = paperdecoding.gettextpages(path)
    pages = [page for page in pages if page is not None]
    outpath = path.replace('\\','_txt\\').replace('pdf','txt')
    paperdecoding.pages_totext(pages,outpath)

# Parse and tidy questions #


In [131]:
all_papers = [paperdecoding.pagesfromtext(x) for x in glob.glob('sa2papers_txt/*[0-9].txt')]

In [132]:
paperdates = [x[14:20] for x in glob.glob('sa2papers_txt/*[0-9].txt')]

Regex out the questions - I'm not sure if sa2 has any single-part questions (ie. questions that don't have a [total x] at the end).

In [133]:
qs = [re.findall('\n\d .+?\[Total \d+\]', '\n'.join(['']+p[1:]), re.DOTALL) for p in all_papers]
qs[25].pop(3)
qs[25].pop(1)
marks = [[float(re.findall('\[Total (\d+?)\]',q)[0]) for q in q_i] for q_i in qs]
qs[13].append(re.findall('\n3 .+?\[\d+\]', '\n'.join(['']+all_papers[13][1:]), re.DOTALL)[0])
marks[13].append(10)
bd = [[[float(mark) for mark in re.findall('\[(\d+?)\]',q)] for q in q_i] for q_i in qs]
bd[26][0]=[3.0, 6.0, 10.0, 8.0, 7.0, 6.0, 8.0]
qnos = [[int(s[1]) for s in q_i] for q_i in qs]

In [134]:
[[sum(x) for x in y] for y in bd]

[[32.0, 68.0],
 [26.0, 30.0, 44.0],
 [55.0, 45.0],
 [30.0, 45.0, 25.0],
 [68.0, 32.0],
 [70.0, 30.0],
 [43.0, 34.0, 23.0],
 [75.0, 25.0],
 [48.0, 52.0],
 [27.0, 29.0, 44.0],
 [57.0, 43.0],
 [35.0, 28.0, 37.0],
 [67.0, 33.0],
 [39.0, 51.0, 10.0],
 [37.0, 63.0],
 [50.0, 50.0],
 [60.0, 24.0, 16.0],
 [48.0, 52.0],
 [48.0, 52.0],
 [66.0, 34.0],
 [44.0, 56.0],
 [50.0, 50.0],
 [50.0, 50.0],
 [60.0, 40.0],
 [44.0, 56.0],
 [44.0, 56.0],
 [48.0, 52.0],
 [48.0, 52.0],
 [34.0, 66.0],
 [53.0, 47.0],
 [31.0, 29.0, 40.0],
 [29.0, 28.0, 43.0],
 [21.0, 29.0, 50.0]]

In [135]:
for i, q_i in enumerate(qs):
    for j, q_ij in enumerate(qs[i]):
        if marks[i][j] != sum(bd[i][j]):
            print(i,j)

In [136]:
q_dictlist = [{'full_date': paperdates[i],
'question_no': qnos[i][j],
'total_marks': marks[i][j],
'breakdown': bd[i][j],
'question_text': q_ij} for i, q_i in enumerate(qs) for j, q_ij in enumerate(qs[i])]

In [137]:
qdf = pd.DataFrame(q_dictlist)
qdf

,full_date,question_no,total_marks,breakdown,question_text
0,200504,1,32.0,"[6.0, 8.0, 18.0]",\n1 A UK life insurance company sells mainly ...
1,200504,2,68.0,"[5.0, 11.0, 5.0, 9.0, 12.0, 26.0]",\n2 Company A is a small UK mutual life insur...
2,200509,1,26.0,"[12.0, 6.0, 8.0]",\n1 A UK bank is about to launch an equity re...
3,200509,2,30.0,"[16.0, 14.0]",\n2 A UK life insurance company currently sel...
4,200509,3,44.0,"[15.0, 6.0, 23.0]",\n3 A UK mutual life insurance company distri...
...,...,...,...,...,...
71,202009,2,28.0,"[3.0, 3.0, 4.0, 7.0, 11.0]",\n2 \n(i) State the three lines of defence a ...
72,202009,3,43.0,"[13.0, 17.0, 13.0]",\n3 \nA small life insurance company runs a wi...
73,202104,1,21.0,"[6.0, 5.0, 10.0]",\n1 A life insurance company sells a unit-lin...
74,202104,2,29.0,"[3.0, 14.0, 6.0, 6.0]",\n2 A life insurance company started writing ...


# Parse and tidy answers

In [2]:
all_reports = [paperdecoding.pagesfromtext(x) for x in glob.glob('sa2papers_txt/*[0-9]report.txt')]

In [4]:
#print(all_reports[0][1])

In [139]:
#reportdates = [paperdecoding.getdatefrompaper(report) for report in all_reports]
reportdates = [x[14:20] for x in glob.glob('sa2papers_txt/*[0-9]report.txt')]
print(reportdates)

['200504', '200509', '200604', '200609', '200704', '200709', '200804', '200809', '200904', '200909', '201004', '201009', '201104', '201109', '201204', '201209', '201304', '201309', '201404', '201409', '201504', '201509', '201604', '201609', '201704', '201709', '201804', '201809', '201904', '201909', '202004', '202009', '202104']


In [111]:
rqnos = [re.split(r'\nQ?(\d) ', ''.join(report))[1::2] for report in all_reports]
rqs = [re.split(r'\nQ?(\d) ', ''.join(report))[2::2] for report in all_reports]

In [112]:
def answerglue(i,j,n=1):
    """Utility function that glues answers back together where they have been incorrectly split up by the RE"""
    for dummy in range(n):
        rqs[i][j] = rqs[i][j] + '\n' + rqnos[i][j+1] + ' ' + rqs[i][j+1]
        rqnos[i].pop(j+1)
        rqs[i].pop(j+1)
answerglue(0,1,3)
answerglue(6,1)
answerglue(9,0)
answerglue(11,0,2)
answerglue(14,0)
answerglue(15,0)
answerglue(17,0,2)
answerglue(30,0)
answerglue(32,0,2)

In [114]:
for i,x in enumerate(rqnos):
    print(i,x)

0 ['1', '2']
1 ['1', '2', '3']
2 ['1', '2']
3 ['1', '2', '3']
4 ['1', '2']
5 ['1', '2']
6 ['1', '2', '3']
7 ['1', '2']
8 ['1', '2']
9 ['1', '2', '3']
10 ['1', '2']
11 ['1', '2', '3']
12 ['1', '2', '2']
13 ['1', '2', '3']
14 ['1', '2']
15 ['1', '2']
16 ['1', '2', '3']
17 ['1', '2']
18 ['1', '2']
19 ['1', '2']
20 ['1', '2']
21 ['1', '2']
22 ['1', '2']
23 ['1', '2']
24 ['1', '2']
25 ['1', '2']
26 ['1', '2']
27 ['1', '2']
28 ['1', '2']
29 ['1', '2']
30 ['1', '2', '3']
31 ['1', '2', '3']
32 ['1', '2', '3']


In [103]:
print(rqs[17][3])

 (i)  Market risk 
     
    Market risk is the risk that as a result of market movements, a firm may be 
exposed to fluctuations in the income from, or value of, its assets, or the 
amount of its liabilities. 
   
    Sources of market risk include:  
 
   movements in interest rates   
   market value of equities   
   movements in exchange rates   
Page 7 Subject SA2 (Life Insurance Specialist Applications)   Examiners  Report, September 2013 
    None of these sources of risk is independent of the others.   
     
    The company is exposed to these risks mainly through the guarantees offered 
on this product. Market movements could result in the policyholders  fund 
values falling. In this case, fund management charge income will be lower 
than expected and the guarantee income charge may be lower than expected  if 
the charge is a percentage of the fund. If policyholder fund values fall there is 
also a risk that the fund cannot support the guaranteed withdrawal benefit 
amount. 

In [122]:
with pdfplumber.open(glob.glob('sa2papers/*[0-9]report.pdf')[9]) as pdf:
    i=3
    temp_pages = pdf.pages
    #im = temp_pages[i].to_image(resolution=70)
    wdf = pd.DataFrame(temp_pages[i].extract_words())
    wdf['height'] = wdf['bottom'] - wdf['top']
    im.draw_rects(wdf.loc[wdf.height==15], fill=(250,250,100,50), stroke=(0,0,0,100))
    print(wdf.loc[wdf.height==15])
im = temp_pages[i].to_image(resolution=70)# + temp_pages[i+1].to_image(resolution=70)
np.array(im)


Empty DataFrame
Columns: [text, x0, x1, top, bottom, upright, direction, height]
Index: []


array(<pdfplumber.display.PageImage object at 0x00000211825A2790>,
      dtype=object)

In [9]:
glob.glob('sa2papers/*[0-9]report.pdf')[0]

'sa2papers\\200504report.pdf'

In [140]:
r_dictlist = []
for i, rdt in enumerate(reportdates):
    for j,q in enumerate(rqnos[i]):
        r_dictlist.append({'full_date': rdt,
        'question_no': int(q),
        'answer_text': rqs[i][j]})
adf = pd.DataFrame(r_dictlist)
#adf

In [145]:
print(qdf.full_date.unique(),adf.full_date.unique())

['200504' '200509' '200604' '200609' '200704' '200709' '200804' '200809'
 '200904' '200909' '201004' '201009' '201104' '201109' '201204' '201209'
 '201304' '201309' '201404' '201409' '201504' '201509' '201604' '201609'
 '201704' '201709' '201804' '201809' '201904' '201909' '202004' '202009'
 '202104'] ['200504' '200509' '200604' '200609' '200704' '200709' '200804' '200809'
 '200904' '200909' '201004' '201009' '201104' '201109' '201204' '201209'
 '201304' '201309' '201404' '201409' '201504' '201509' '201604' '201609'
 '201704' '201709' '201804' '201809' '201904' '201909' '202004' '202009'
 '202104']


In [146]:
pd.merge(qdf,adf, how = 'outer', indicator = True)[["full_date","_merge"]].drop_duplicates()

,full_date,_merge
0,200504,both
2,200509,both
5,200604,both
7,200609,both
10,200704,both
12,200709,both
14,200804,both
17,200809,both
19,200904,both
21,200909,both


In [147]:
fulldf = pd.merge(qdf,adf)
fulldf[:7]

,full_date,question_no,total_marks,breakdown,question_text,answer_text
0,200504,1,32.0,"[6.0, 8.0, 18.0]",\n1 A UK life insurance company sells mainly ...,(i) The company should consider the extent t...
1,200504,2,68.0,"[5.0, 11.0, 5.0, 9.0, 12.0, 26.0]",\n2 Company A is a small UK mutual life insur...,(i) (a) These regulations give powers to so...
2,200509,1,26.0,"[12.0, 6.0, 8.0]",\n1 A UK bank is about to launch an equity re...,(i) \nThe main risk is that the value of the ...
3,200509,2,30.0,"[16.0, 14.0]",\n2 A UK life insurance company currently sel...,(i) The main factors to consider will be: ...
4,200509,3,44.0,"[15.0, 6.0, 23.0]",\n3 A UK mutual life insurance company distri...,(i) Impact on Free Assets: \nThe reduction ...
5,200604,1,55.0,"[13.0, 5.0, 7.0, 9.0, 8.0, 13.0]",\n1 A UK life insurance company has three fun...,(i) Shareholder Fund \nThis could simply ...
6,200604,2,45.0,"[17.0, 8.0, 10.0, 10.0]",\n2 A UK mutual life insurance company write...,(i) Asset/liability matching \nThe basic...


In [148]:
fulldf.to_csv('fulldf.csv')

In [149]:
fulldf_copy = pd.read_csv('fulldf.csv', index_col=0)
fulldf_copy[:7]

,full_date,question_no,total_marks,breakdown,question_text,answer_text
0,200504,1,32.0,"[6.0, 8.0, 18.0]",\n1 A UK life insurance company sells mainly ...,(i) The company should consider the extent t...
1,200504,2,68.0,"[5.0, 11.0, 5.0, 9.0, 12.0, 26.0]",\n2 Company A is a small UK mutual life insur...,(i) (a) These regulations give powers to so...
2,200509,1,26.0,"[12.0, 6.0, 8.0]",\n1 A UK bank is about to launch an equity re...,(i) \nThe main risk is that the value of the ...
3,200509,2,30.0,"[16.0, 14.0]",\n2 A UK life insurance company currently sel...,(i) The main factors to consider will be: ...
4,200509,3,44.0,"[15.0, 6.0, 23.0]",\n3 A UK mutual life insurance company distri...,(i) Impact on Free Assets: \nThe reduction ...
5,200604,1,55.0,"[13.0, 5.0, 7.0, 9.0, 8.0, 13.0]",\n1 A UK life insurance company has three fun...,(i) Shareholder Fund \nThis could simply ...
6,200604,2,45.0,"[17.0, 8.0, 10.0, 10.0]",\n2 A UK mutual life insurance company write...,(i) Asset/liability matching \nThe basic...


In [150]:
i = 40
print(fulldf.question_text[i])
print('-'*70)
print(fulldf.answer_text[i])


2  (i)   Describe how a single premium unit-linked investment bond would be taxed 
from the viewpoint of a UK policyholder.  (You may assume that such a 
policy is  non-qualifying  for taxation purposes.)  [4] 
 
  It is 2015, and the following is an extract from the tax computations of a UK 
proprietary life insurance company. 
 
  2013  2014 
   k   k 
 
BLAGAB  I (excluding dividend income)  600  900 
    Share of dividend income  -  - 
    E (including any carried forward XSE)  500  300 
    Life Assurance Trade Profits  200  250 
     
OLTB   Premiums  170  260 
    Investment income  40  80 
    Increase in value of assets  20  40 
    Expenses  30  25 
    Benefits paid  30  40 
    Value of liabilities at the start of the year  600  700 
    Value of liabilities at the end of the year  700  1,000 
    Loss brought forward  20  - 
 
  (ii)  Determine the company s tax liability for the years 2013 and 2014, explaining 
your workings.  (You may assume that the policyholder rate o